In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import geopandas as gpd
import pandas as pd
from copy import deepcopy
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon, LineString, MultiLineString
from shapely.ops import unary_union
import numpy as np
from shapely.wkt import loads

In [3]:
gdf_hole = pd.read_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/マンホール/yokohama_2018_2022.csv', encoding='cp932').filter(['人孔番号', 'geometry_x'])


def parse_point(point_str):
    try:
        #文字列になっているから必要な数値だけ取得（POINT Z）を削除
        point_str = point_str[9:-2]

        #空白で分けx, yに代入
        x_str, y_str = point_str.strip().split()
        #ポイントに変換
        return Point(float(x_str), float(y_str))

        #エラーの場合は欠損地を入力
    except ValueError:
        return np.nan

#各行に対して関数を用いるときはapplyを用いる。Seriesだとエラーが起こることが多い
gdf_hole['geometry_x'] = gdf_hole['geometry_x'].apply(parse_point)


#dataframeをgeodataframeに変換
gdf_hole = GeoDataFrame(gdf_hole, geometry='geometry_x')

<ipython-input-3-c358443521cf>:1: DtypeWarning: Columns (94,98,101,115) have mixed types. Specify dtype option on import or set low_memory=False.
  gdf_hole = pd.read_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/マンホール/yokohama_2018_2022.csv', encoding='cp932').filter(['人孔番号', 'geometry_x'])


In [4]:
gdf_factory_1 = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/工場用地/L05-1-09_14_GML/L05-1-09_14-g_IndustrialComplex.shp')
gdf_factory_2 = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/工場用地/L05-2-09_14_GML/L05-2-09_14-g_IsolatedIndustrialSite.shp')

#データフレームを結合
gdf_factory = pd.concat([gdf_factory_1, gdf_factory_2], ignore_index=True)

gdf_factory = gdf_factory[['L05_001', 'L05_002', 'L05_003', 'L05_004', 'L05_005', 'L05_006','L05_007', 'L05_008', 'L05_009', 'L05_010', 'L05_011', 'L05_012','L05_013', 'L05_014', 'L05_015', 'L05_016', 'geometry']]
new_columns = ['Industrial_Site_ID',
'industrial_Site_name',
'administrative_district_code',
'name_of_city',
'Waterfront_inland_code',
'special_remarks',
'District_Name',
'Business_Entity_Name',
'Project_Completion_Year',
'Overall_Development_Area',
'Industrial_land_area',
'Unutilized_area',
'Zoning_Code',
'Number_of_companies',
'Company_Name',
'Type_of_Business',
'geometry']

columns = gdf_factory.columns.to_list()

#カラム名のペアを作成
columns_dir = {i: j for i, j in zip(columns, new_columns)}

#カラム名を変更
gdf_factory.rename(columns=columns_dir, inplace=True)

In [5]:
if gdf_hole.crs is None:
    # 座標系をEPSG:6677に設定
    gdf_hole.set_crs(epsg=6677, inplace=True)

In [6]:
#座標系を変更
src_proj = 4326 # 変換前の座標系を指定
dst_proj = 6677  # 変換後の座標系を指定

gdf_factory.crs = f'epsg:{src_proj}'  # 変換前座標を指定
gdf_factory = gdf_factory.to_crs(epsg=dst_proj)  # 変換後座標に変換

In [7]:
#空間結合した後にgeomeryを加えられるように連番を追加
number = [i for i in range(len(gdf_factory))]
gdf_factory['number'] = number

In [8]:
#空間結合をする
result = gpd.sjoin_nearest(gdf_hole, gdf_factory, how="left", distance_col="distance_factory")
#空間結合された道路のgeometryを追加
result = pd.merge(result, gdf_factory[['number', 'geometry']], on='number')
# #geometryの名前を変更
result.rename(columns={'geometry':'nearest_factory'}, inplace=True)
#メモリの解放
# del gdf_factory

In [11]:
#numberはいらないのでdrop
result.drop(columns=['index_right', 'number'], axis=1, inplace=True)

In [35]:
result_columns = result.columns.to_list()
result[result_columns][(result['industrial_Site_name'] == '第一次戸塚中小企業団地') | (result['industrial_Site_name'] == '第二次戸塚中小企業団地')][5000:10000].to_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/横浜市/横浜市　最近傍工場/横浜市_主要工場_確認用_v2.csv', index=False)

In [38]:
result[result['人孔番号'] == 2420130012][['人孔番号', 'industrial_Site_name','distance_factory']]

,人孔番号,industrial_Site_name,distance_factory
92432,2420130012,根岸湾臨海工業地帯,491.958895
92433,2420130012,根岸湾臨海工業地帯,491.958895


In [33]:
result[result_columns][(result['industrial_Site_name'] == '第一次戸塚中小企業団地') | (result['industrial_Site_name'] == '第二次戸塚中小企業団地')]

,人孔番号,geometry_x,Industrial_Site_ID,industrial_Site_name,administrative_district_code,name_of_city,Waterfront_inland_code,special_remarks,District_Name,Business_Entity_Name,Project_Completion_Year,Overall_Development_Area,Industrial_land_area,Unutilized_area,Zoning_Code,Number_of_companies,Company_Name,Type_of_Business,distance_factory,nearest_factory
252437,2313200043,POINT (-30081.617 -63354.525),1452400,第一次戸塚中小企業団地,14110,横浜市戸塚区,2,NaN,NaN,（財）横浜市開発公社,1964,13.8,0,0,2,21,NaN,NaN,2384.181491,"POLYGON ((-27987.106 -64585.042, -27983.094 -6..."
252438,2313210006,POINT (-29828.111 -63391.216),1452400,第一次戸塚中小企業団地,14110,横浜市戸塚区,2,NaN,NaN,（財）横浜市開発公社,1964,13.8,0,0,2,21,NaN,NaN,2144.134543,"POLYGON ((-27987.106 -64585.042, -27983.094 -6..."
252439,2313210052,POINT (-30035.817 -63351.966),1452400,第一次戸塚中小企業団地,14110,横浜市戸塚区,2,NaN,NaN,（財）横浜市開発公社,1964,13.8,0,0,2,21,NaN,NaN,2344.878102,"POLYGON ((-27987.106 -64585.042, -27983.094 -6..."
252440,2313210139,POINT (-29908.155 -63361.007),1452400,第一次戸塚中小企業団地,14110,横浜市戸塚区,2,NaN,NaN,（財）横浜市開発公社,1964,13.8,0,0,2,21,NaN,NaN,2228.643580,"POLYGON ((-27987.106 -64585.042, -27983.094 -6..."
252441,2313210140,POINT (-29884.757 -63366.862),1452400,第一次戸塚中小企業団地,14110,横浜市戸塚区,2,NaN,NaN,（財）横浜市開発公社,1964,13.8,0,0,2,21,NaN,NaN,2205.401969,"POLYGON ((-27987.106 -64585.042, -27983.094 -6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305459,2114140029,POINT (-27285.276 -59901.965),1452500,第二次戸塚中小企業団地,14110,横浜市戸塚区,2,NaN,NaN,（財）横浜市開発公社,1966,11.3,0,0,2,61,NaN,NaN,4481.727965,"POLYGON ((-27373.626 -64425.753, -27361.882 -6..."
305460,2114040005,POINT (-27268.357 -59859.970),1452500,第二次戸塚中小企業団地,14110,横浜市戸塚区,2,NaN,NaN,（財）横浜市開発公社,1966,11.3,0,0,2,61,NaN,NaN,4524.103034,"POLYGON ((-27373.626 -64425.753, -27361.882 -6..."
305461,2114040006,POINT (-27260.503 -59872.769),1452500,第二次戸塚中小企業団地,14110,横浜市戸塚区,2,NaN,NaN,（財）横浜市開発公社,1966,11.3,0,0,2,61,NaN,NaN,4511.510910,"POLYGON ((-27373.626 -64425.753, -27361.882 -6..."
305462,2114140030,POINT (-27236.382 -59917.719),1452500,第二次戸塚中小企業団地,14110,横浜市戸塚区,2,NaN,NaN,（財）横浜市開発公社,1966,11.3,0,0,2,61,NaN,NaN,4467.292781,"POLYGON ((-27373.626 -64425.753, -27361.882 -6..."
